In [19]:
import parse_tfl_json
import google

In [20]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime 
where tfl_message = 'ok' 
"""
df = pd.read_sql(sql, conn)

In [21]:
from process_journeys import get_london_icscodes, get_cycle_time_mem, add_cycle_and_total_time, remove_nonlondon_journeys
from google import get_cycle_info
import parse_tfl_json

In [22]:
import json 

# Get list of cycle times required
cycletimes_needed = []


for r in df.iterrows():
    index = r[0]
    row = r[1]

    j1 = row["tfl_response"]
    j2 = json.loads(j1)
    
    
    journeys = j2["journeys"]
    # Keep only journeys that end in central London

    try:
        for journey in journeys:
            legs = journey["legs"]
            journey["legs"] = parse_tfl_json.remove_non_national_rail(legs)

        journeys = remove_nonlondon_journeys(journeys)

        for journey in journeys:
            tfl_dest = row["tfl_dest"]
            lat = journey["legs"][-1]["arrivalPoint"]["lat"]
            lng = journey["legs"][-1]["arrivalPoint"]["lon"]
            name = journey["legs"][-1]["arrivalPoint"]["commonName"]

            to_add = {"tfl_dest": tfl_dest, "lat": lat, "lng": lng, "name": name}
            cycletimes_needed.append(to_add)
    except Exception as e:
        pass



In [24]:
len(pd.DataFrame(cycletimes_needed).drop_duplicates())

84

In [4]:


for r in df.iterrows():
    index = r[0]
    row = r[1]

    j1 = row["tfl_response"]
    j2 = json.loads(j1)
    
    
    journeys = j2["journeys"]
    # Keep only journeys that end in central London


    for journey in journeys:
        legs = journey["legs"]
        journey["legs"] = parse_tfl_json.remove_non_national_rail(legs)

    journeys = remove_nonlondon_journeys(journeys)


    journeys = add_cycle_and_total_time(journeys, row["tfl_dest"])

    journeys = sorted(journeys, key = lambda x: x["total_time"])

    journey = journeys[0]
    legs = journey["legs"]

    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary

    df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
    df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)

    df.loc[index, "depart_lat"] = lat_lng["depart"]["lat"]
    df.loc[index, "depart_lng"] = lat_lng["depart"]["lng"]
    df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
    df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]

    df.loc[index, "cycle_minutes"] = journey["cycle_minutes"]
    df.loc[index, "cycle_miles"] = journey["cycle_miles"]
    df.loc[index, "total_journeytime"] = journey["cycle_minutes"] + travel_time_minutes

#     except Exception as e:
#         df.loc[index, "error"] = "Parse problem"
    
    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict", "journey_time", "london_mainline_station_ics", "london_mainline_station_text"]]
df2 = df[cols]

ConnectionError: HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/distancematrix/json?units=imperial&origins=51.528952007200004,-0.1350003194&destinations=SW1H9AJ (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x10d56c668>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
df2


In [ ]:
df3 = df2.sort_values(["station_name", "total_journeytime"]).drop_duplicates("station_name")

In [ ]:
df_arrivals = df2.drop_duplicates("final_arrival")[["final_arrival", "arrive_lat", "arrive_lng"]]


In [ ]:
df_arrivals = df_arrivals[["final_arrival", "cycle_miles_london", "cycle_minutes_london"]]
df4 = df3.merge(df_arrivals, how = "left", left_on = "final_arrival", right_on = "final_arrival")

In [ ]:
keep = ["station_name", "natrail_journey_summary", "depart_lat", "depart_lng", "natrail_journey_minutes","cycle_minutes_london"]
df4 = df4[keep]
df4["total_minutes"] = df4["cycle_minutes_london"] + df4["natrail_journey_minutes"]


In [ ]:
# Want to update all the names to be cw specific 
df5 = df4.rename(columns={"depart_lat" : "lat", "depart_lng": "lng", "station_name": "name"})

In [ ]:
df5.to_csv("cw_to_map.csv", encoding="utf-8", index=False)

In [ ]:
df5.columns

In [ ]:
# To do - pick best journey time by total, not just the time on the train.  
# We can filter out suggested journeys where the lat lng is outside of central London

In [ ]:
journeys[1]